In [40]:
#Add all the imports here
import time
import logging
import warnings
import pandas as pd
import yfinance as yf
from dataclasses import dataclass
from datetime import datetime, timedelta

import numpy as np
from itertools import combinations
import itertools
import plotly.graph_objects as go

In [30]:
# Silence yfinance logs
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

# Silence warnings
warnings.filterwarnings("ignore")

In [31]:
def get_ticker_list():
    # --- Auth (Colab only) ---
    from google.colab import auth
    auth.authenticate_user()

    import gspread
    import pandas as pd
    from google.auth import default

    # --- Authorize ---
    creds, _ = default()
    gc = gspread.authorize(creds)

    # --- Read sheet ---
    sheet = gc.open_by_url(
        "https://docs.google.com/spreadsheets/d/1SAS2_nj_BN22vsYMbpG79MgbiBLI1YaVkRejYMWe3hE/edit"
    ).sheet1

    # Pull only first column (much faster than get_all_records)
    values = sheet.col_values(1)

    # Skip header
    s = (
        pd.Series(values[1:])
        .dropna()
        .astype(str)
        .str.strip()
    )

    tickers = (
        s[s.ne("")]
        .where(s.str.endswith(".NS"), s + ".NS")
        .tolist()
    )

    return tickers


In [32]:
#Yfinance helpers

IST_TZ = "Asia/Kolkata"

def fix_ist(df: pd.DataFrame) -> pd.DataFrame:
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC")
    return df.tz_convert(IST_TZ)

def extract_ticker(df: pd.DataFrame, ticker: str) -> pd.DataFrame | None:
    if df is None or df.empty:
        return None

    # Fast path: single-level columns
    if df.columns.nlevels == 1:
        return df

    # MultiIndex case
    if ticker in df.columns.get_level_values(0):
        return df[ticker]

    return None

def get_single_symbol_day(symbol: str, date: str, interval: str) -> pd.DataFrame | None:
    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)

    df = yf.download(
        symbol,
        start=start,
        end=end,
        interval=interval,
        progress=False,
        threads=False
    )

    if df.empty:
        return None

    df = extract_ticker(df, symbol)
    return fix_ist(df)

def get_multiple_symbols_day(
    symbols: list[str],
    date: str,
    interval: str,
    batch_size: int = 50,
    throttle_sec: float = 1.0
) -> pd.DataFrame | None:

    start_ts = time.perf_counter()

    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)

    all_batches = []

    for i in range(0, len(symbols), batch_size):
        batch = symbols[i:i + batch_size]

        df = yf.download(
            batch,
            start=start,
            end=end,
            interval=interval,
            group_by="ticker",
            progress=False,
            threads=False
        )

        if not df.empty:
            all_batches.append(df)

        # Required for intraday throttling
        if i + batch_size < len(symbols):
            time.sleep(throttle_sec)

    if not all_batches:
        return None

    result = pd.concat(all_batches, axis=1)
    result = fix_ist(result)

    elapsed = time.perf_counter() - start_ts
    print(f"'get_multiple_symbols_day': Done in {elapsed:.2f}s")

    return result

In [33]:
# -------------------- Figure creation --------------------

def create_candlestick_figure(ticker_symbol: str, date: str) -> go.Figure:
    fig = go.Figure(
        layout=dict(
            title=f"{ticker_symbol} Candlestick Chart",
            xaxis_title=f"Date: {date}",
            yaxis_title="Price",
            dragmode="pan",
            xaxis=dict(
                type="category",
                rangeslider=dict(visible=False),
                tickmode="array",
                showticklabels=False,
                fixedrange=False,
            ),
            yaxis=dict(fixedrange=False),
        )
    )
    return fig


# -------------------- Candlestick --------------------

def update_candlestick_data(
    fig: go.Figure,
    historical_data: pd.DataFrame
) -> go.Figure:
    fig.add_trace(
        go.Candlestick(
            x=historical_data.index,
            open=historical_data["Open"],
            high=historical_data["High"],
            low=historical_data["Low"],
            close=historical_data["Close"],
            increasing_line_color="green",
            decreasing_line_color="red",
        )
    )
    return fig


# -------------------- Helpers --------------------

def add_hline(
    fig: go.Figure,
    value: float,
    annot: str,
    color: str = "blue",
    annot_pos: str = "top left",
) -> go.Figure:
    fig.add_hline(
        y=value,
        line=dict(color=color, width=1, dash="dash"),
        annotation_text=annot,
        annotation_position=annot_pos,
    )
    return fig


def add_vzone(
    fig: go.Figure,
    start_dt,
    end_dt,
    color: str = "rgba(255,0,0,0.12)",
) -> go.Figure:
    fig.add_vrect(
        x0=start_dt,
        x1=end_dt,
        fillcolor=color,
        line_width=0,
        layer="below",
    )
    return fig


def add_marker(
    fig: go.Figure,
    time,
    price: float,
    text: str,
) -> go.Figure:
    fig.add_annotation(
        x=time,
        y=price,
        text=text,
        showarrow=True,
        arrowhead=2,
        arrowsize=1.2,
        arrowcolor="crimson",
        ax=-40,
        ay=-40,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="crimson",
        borderwidth=1,
        font=dict(size=12, color="black"),
        xref="x",
        yref="y",
    )
    return fig


def add_header(fig: go.Figure, text: str) -> go.Figure:
    fig.add_annotation(
        text=text,
        xref="paper",
        yref="paper",
        x=0.5,
        y=1.06,
        showarrow=False,
        font=dict(size=16),
    )
    return fig


Above are refactored

In [34]:
@dataclass
class TradeStats:
    win: int = 0
    loss: int = 0
    pnl: float = 0.0
    tot_trade: int = 0

    def update(self, result, current_pct):
        if result == "win":
            self.win += 1
        else:
            self.loss += 1

        self.pnl += current_pct
        self.tot_trade += 1

    def merge(self, other):
        self.win += other.win
        self.loss += other.loss
        self.pnl += other.pnl
        self.tot_trade += other.tot_trade

def print_summary(stats, title="Summary"):
    print(f"\n{title}")
    print("-" * 30)
    print(f"Trades: {stats.tot_trade}")
    print(f"Win: {stats.win} | Loss: {stats.loss}")
    print(f"PnL: {round(stats.pnl, 2)}%")



In [35]:
def maybe(fig, show_fig, fn, *args, **kwargs):
    if not show_fig:
        return fig
    return fn(fig, *args, **kwargs)

def fig_data_support_resistance(fig: go.Figure,data,support,resistance):
  fig = update_candlestick_data(fig, historical_data=data)
  fig = add_hline(fig,support,"support")
  fig = add_hline(fig,resistance,"resistance")
  return fig

def fig_start_zone(fig: go.Figure,date):
  start_dt = pd.Timestamp(f"{date} 10:10:00").tz_localize("Asia/Kolkata")
  end_dt   = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")
  fig = add_vzone(fig, start_dt, end_dt)
  return fig

def fig_entry_target_stop(fig: go.Figure,position,entry,entry_index,stop,target):
  fig = add_marker(fig, entry_index, entry, f"Entry({position}):{entry}")
  fig = add_hline(fig, stop, f"Stoploss:{stop}", "red", "top right")
  fig = add_hline(fig, target, f"Target:{target}", "green", "top right")
  return fig


In [41]:
def compute_trade_result(entry, exit_price, position):
    return round(
        ((entry - exit_price) / entry) * 100 if position == "Short"
        else ((exit_price - entry) / entry) * 100,
        2
    )



def update_trade_result(fig, stats, result, entry, exit_price, index, position, show_fig=False):
    current_pct = compute_trade_result(entry, exit_price, position)
    stats.update(result, current_pct)


    fig = maybe(fig, show_fig, add_marker, index, exit_price, result.capitalize())
    fig = maybe(fig, show_fig, add_header,
                f"Current: {result.capitalize()} | PnL:{current_pct}%  ||  "
                f"Total: Win:{stats.win} Loss:{stats.loss}  PnL:{stats.pnl}%")

    print(f"Current: {result.capitalize()} | PnL:{current_pct}%  ||  "
                f"Total: Win:{stats.win} Loss:{stats.loss}  PnL:{stats.pnl}%")

    return fig


def evaluate_trade(fig, data, start_i, entry, stop, target,
                   position, stats, show_fig=False):

    i = start_i

    while i < len(data):
        row = data.iloc[i]
        index = row.name
        high, low = row.High, row.Low

        if low <= target <= high:
            return update_trade_result(
                fig, stats, "win", entry, target,
                index, position, show_fig)

        if low <= stop <= high:
            return update_trade_result(
                fig, stats, "loss", entry, stop,
                index, position, show_fig)

        i += 1

    return fig


In [36]:
def strategy1(ticker_symbol, date, data_5m, show_fig=False):
  stats = TradeStats()
  fig = create_candlestick_figure(ticker_symbol, date) if show_fig else None

  # --------- LEVELS (Higher TF) ---------
  operating_range = data_5m[:12]
  support    = operating_range["High"].max()
  resistance = operating_range["Low"].min()

  # --------- LOWER TF DATA ---------
  data = data_5m

  fig = maybe(fig, show_fig, fig_start_zone, date)
  fig = maybe(fig, show_fig, fig_data_support_resistance, data, support, resistance)

  start_tf = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")

  # --------- STATE ---------
  short_state = {"breakout": False, "risk": None}
  long_state  = {"breakout": False, "risk": None}

  # --------- ITERATION ---------
  i = 0
  while i < len(data):
    row = data.iloc[i]
    index = row.name

    if index < start_tf:
      i += 1
      continue

    close, high, low = row.Close, row.High, row.Low

    # =========== SHORT SETUP ==================
    if not short_state["breakout"]:
      if close > support:
        short_state.update({"breakout": True, "risk": high})
        fig = maybe(fig, show_fig, add_marker, index, close, "Breakout (Short)")

    else:
      short_state["risk"] = max(short_state["risk"], high)

      if close < support:
        entry = close
        stop = short_state["risk"]
        target = entry + (entry - stop) * 2

        fig = maybe(fig, show_fig, fig_entry_target_stop, "Short", entry, index, stop, target)

        fig = evaluate_trade(fig, data, i + 1, entry, stop, target, "Short", stats, show_fig)

        short_state = {"breakout": False, "risk": None}
        i += 1
        #continue

        break   #single trade per day

    # ================= LONG =================
    if not long_state["breakout"]:
      if close < resistance:
        long_state["breakout"] = True
        long_state.update({"breakout": True, "risk": low})
        fig = maybe(fig, show_fig, add_marker, index, close, "Breakout (Long)")

    else:
      long_state["risk"] = min(long_state["risk"], low)

      if close > resistance:
        entry = close
        stop = long_state["risk"]
        target = entry - (stop - entry) * 2

        fig = maybe(fig, show_fig, fig_entry_target_stop,"Long", entry, index, stop, target)

        fig = evaluate_trade(fig, data, i + 1, entry, stop, target, "Long", stats, show_fig)

        long_state = {"breakout": False, "risk": None}
        i += 1
        #continue

        break   #single trade per day

    i += 1

  if show_fig and fig is not None:
    fig.show(config={"scrollZoom": True})

  return stats


In [38]:
def get_dates(days_back):
    today = pd.Timestamp.now(tz=tz).normalize()
    start = today - pd.Timedelta(days=days_back - 1)

    dates = pd.date_range(start=start, end=today, freq="D", tz=IST_TZ)

    date_list = []
    for dt in dates:
        date_str = dt.strftime("%Y-%m-%d")
        date_list.append(date_str)

    return date_list

In [ ]:
def run_backtest_strategy1(ticker_symbols, dates, show_fig=False):
  if not ticker_symbols:
        return
  if not dates:
        return

  for date in dates:
    data_5m = get_multiple_symbols_day(ticker_symbols, date, "5m")
    if data_5m is None:
      print(f"No data found for {date}")
      return
    print(f"Running for {date}")
    for idx, symbol in enumerate(ticker_symbols, 1):
      print(f"Running {symbol} ({idx}/{total_symbols})")
      ex_5m = extract_ticker(data_5m, symbol)
      if ex_5m is None:
        print(f"No data found for {symbol}")
        continue
      stats = strategy1(symbol, date, ex_5m, show_fig)
      print_summary(stats, f"Summary for {date/symbol}")


In [ ]:
#Main entry

# For testing single
#ticker_symbol = "HDFCBANK.NS"
#date = "2025-12-19"
#data_1h = get_single_symbol_day("HDFCBANK.NS","2025-12-19","1h")
#data_5m = get_single_symbol_day("HDFCBANK.NS","2025-12-19","5m")
#fig = strategy1(ticker_symbol, date, data_1h, data_5m)
#fig.show(config={"scrollZoom": True})

run_backtest_strategy1(get_ticker_list()[0:2], get_dates(1), True)

# Run Multi ticker per day
#date = "2025-12-29"
#ticker_symbol = ["HDFCBANK.NS","TITAN.NS"]
#ticker_symbol = get_ticker_list()
#run_multi_tickers_per_day(ticker_symbol, date, True)

# Run multi ticker multiday
#ticker_symbol = ["HDFCBANK.NS","TITAN.NS"]
#ticker_symbol = get_ticker_list()[0:20]
#run_day_range(ticker_symbol,1, True)